In [1]:
# Importamos las librerías necesarias
import numpy as np
import pandas as pd
import seaborn as snb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
# from sklearn.impute import SimpleImputer
# imp = SimpleImputer(strategy='mean')

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_log_error

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline
pd.options.mode.chained_assignment = None

In [2]:
# Jalamos los datos del dataset
data_train = pd.read_csv('house_train_raw.csv', encoding='utf-8', sep=',')
data_test = pd.read_csv('houses_test_raw.csv', encoding='utf-8', sep=',')
dataf = pd.concat((data_train, data_test)).reset_index(drop=True)


In [ ]:
# Obtenemos inforación del dataset
dataf.info()

In [ ]:
# Obtenemos el número de registros y columnas del dataset
dataf.shape

In [ ]:
# Imprimimos en pantalla los 5 primeros registros
dataf.head(5)

In [ ]:
# Creamos una función que devuleva un dataframe con el conteo y porcentaje de nulos
def porcentaje_nulos(col=''):
  null_counts = pd.DataFrame()
  null_counts['conteo'] = dataf.isnull().sum()
  null_counts = null_counts[null_counts['conteo']>0].reset_index()
  null_counts['porcentaje'] = (null_counts['conteo']/dataf.shape[0] * 100).round(2)
  if col in ['','all']:
    return null_counts
  return null_counts['porcentaje'].loc[null_counts['index'] == col][0]


In [ ]:
porcentaje_nulos()

In [ ]:
# Rellenamos la columna ['MSZoning'] con la moda, ya que el porcentaje de nulos es menos del 1%
dataf['MSZoning']= dataf['MSZoning'].fillna(dataf['MSZoning'].mode()[0])
porcentaje_nulos()

In [ ]:
# Rellenamos la columna 'LotFrontage' con su promedio porque es numérico y el porcentaje de nulos es menos del 17%
dataf['LotFrontage']= dataf['LotFrontage'].fillna(dataf['LotFrontage'].mean())
porcentaje_nulos()

In [ ]:
# Eliminamos la columna, ya que contiene un 93.22 % de nulos y a mi parecer no va a influir mucho en el resultado final
# porcentaje_nulos('Alley') -----> 93.22 %
dataf.drop(['Alley'], axis=1, inplace=True)
porcentaje_nulos()

In [ ]:
# Rellenamos las columnas ['Utilities','Exterior1st','Exterior2nd','MasVnrType','MasVnrArea'] con la moda, ya que el porcentaje de nulos es menos del 1% cada uno.
columnas = ['Utilities','Exterior1st','Exterior2nd','MasVnrType','MasVnrArea']
for columna in columnas:
  dataf[columna] = dataf[columna].fillna(dataf[columna].mode()[0])
porcentaje_nulos()

In [ ]:
# Rellenamos las columnas con la moda ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF'] porque son de tipo objeto
# además los nulos de cada columna no pasan del 3% cada uno.

columnas = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF']
for col in columnas:
  # print(f"porcentaje_nulos('{col}') -----> {porcentaje_nulos(col)} %")
  dataf[col] = dataf[col].fillna(dataf[col].mode()[0])
porcentaje_nulos()

In [ ]:
#  Rellenamos las columnas con la moda ['Electrical','BsmtFullBath','BsmtHalfBath','KitchenQual','Functional','SaleType'] porque es de tipo objeto, además no pasan del 1% cada uno.
columnas = ['Electrical','BsmtFullBath','BsmtHalfBath','KitchenQual','Functional','SaleType']
for columna in columnas:
  dataf[columna] = dataf[columna].fillna(dataf[columna].mode()[0])
porcentaje_nulos()

In [ ]:
# Eliminanos los registros que no contienen valor en el año de construcción en la columna ['GarageYrBlt']
dataf.dropna(subset=['GarageYrBlt'], inplace=True)
porcentaje_nulos()

In [ ]:
# Reemplazamos los valor nulos por el valor NA que significa (No Fireplace)
dataf['FireplaceQu'] = dataf['FireplaceQu'].fillna('NA')
# dataf['FireplaceQu'].value_counts()
porcentaje_nulos()

In [ ]:
# Eliminamos las columnas ['PoolQC','Fence','MiscFeature'], porque sobrepasan el 80% de nulos.
columnas = ['PoolQC','Fence','MiscFeature']
dataf.drop(columns=columnas, axis=1, inplace=True)
porcentaje_nulos()

In [ ]:
# Verificamos si hay duplicados
dataf.duplicated().sum()

In [ ]:
# Volvemos a verificar el número de registros y columnas
dataf.shape

In [ ]:
dataf.info()

In [ ]:
# Recodificamos la columna ['MSSubClass'] para tomarlo en cuenta como categórico y no numérico
dataf['MSSubClass'] = dataf['MSSubClass'].apply(lambda x : "MSSC"+str(x))

In [ ]:
# Seperamos las columnas de tipo Objeto y numérico
col_objets = []
for i in dataf.columns:
    if dataf[i].dtype == object:
        col_objets.append(i)

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
col_numerics = []
for i in dataf.columns:
    if dataf[i].dtype in numeric_dtypes:
        col_numerics.append(i)

In [ ]:
print(f"El número de columas de tipo objeto son: {len(col_objets)}")
print(f"El número de columas de tipo numérico son: {len(col_numerics)}")

In [ ]:
col_objets

In [ ]:
col_numerics

In [ ]:
# Extraemos sólo las columnas que son numéricas
df_final_numerics = dataf[col_numerics]

In [ ]:
# Definimos una función que me permite aplicar Onehotencoder
def category_onehotencoder(cols, df_encoder):
    onehot_encoder = OneHotEncoder(sparse = False)
    df_ = df_final_numerics.copy()
    for col in cols:
        col_encoder = df_encoder[col].values.reshape(-1,1)
        onehot_encoder.fit(col_encoder)
        cols_encoded = onehot_encoder.transform(col_encoder)

        for i, e in enumerate(onehot_encoder.categories_[0]):
            df_[e+"_encode"] = cols_encoded[:,i]
    return df_

In [ ]:
# Asignamos a la variable 'df_final_encode' el resultado del Onehotencoder
df_final_encode = category_onehotencoder(col_objets,dataf)

## Entrenando con modelo Árbol de Decisión (onehotencoder y numéricas)

In [ ]:
# Contamos los registros que contengan valor en la columna ['SalePrice] para diferenciar el dataframe train con el test
conteo_train_ad = len(df_final_encode[(~df_final_encode.SalePrice.isnull())])

In [ ]:
# Separando X_df_train_ad e X_df_test_ad
X_df_train_ad = df_final_encode.iloc[:conteo_train_ad,:]
y_df_train_ad = X_df_train_ad.SalePrice.values

X_df_test_ad = df_final_encode.iloc[conteo_train_ad:,:]
X_df_test_ad_id = X_df_test_ad.iloc[:,:1]

In [ ]:
# Eliminamos la columna ['Id','SalePrice] de ambos dataframes
X_df_train_ad.drop(['Id','SalePrice'], axis=1, inplace=True)
X_df_test_ad.drop(['Id','SalePrice'], axis=1, inplace=True)

In [ ]:
# Creamos un objeto árbol y hacemos el split del dataframe
tree = DecisionTreeClassifier()

X_train_ad, X_test_ad, y_train_ad, y_test_ad = train_test_split(X_df_train_ad, y_df_train_ad, test_size=0.33, random_state=42)

In [ ]:
# Entrenamos el modelo
tree.fit(X_train_ad, y_train_ad)

In [ ]:
# Predecimos sobre nuestro set de entrenamieto
y_train_pred_ad = tree.predict(X_train_ad)

# Predecimos sobre nuestro set de test
y_test_pred_ad = tree.predict(X_test_ad)

In [ ]:
print('La raíz cuadrada del error logarítmico medio (RMSLE) sobre el dataset houses_train_raw es:', mean_squared_log_error(y_test_pred_ad, y_test_ad, squared=False))

In [ ]:
plt.scatter(y_test_pred_ad, y_test_ad, c="r", alpha=0.5)
plt.xlabel("y_test_pred_ad")
plt.ylabel("y_test_ad")
# plt.legend(loc='upper left')
plt.show()

### Predecimos la columna SalePrice del dataset "houses_test_raw.csv"

In [ ]:
y_test_pred_raw = tree.predict(X_df_test_ad)

In [ ]:
pred_test_ad = pd.DataFrame(y_test_pred_raw, columns=['SalePrice'])
X_df_test_ad_id.reset_index(drop=True, inplace=True)
pred_test_ad.reset_index(drop=True, inplace=True)
pred_concat_ad = pd.concat( [X_df_test_ad_id, pred_test_ad], axis=1) 

pred_concat_ad.to_csv("pred_test_ad.csv", index=False, sep=",")

## Usando el modelo "Extreme Gradient Boosting"
https://medium.com/@jboscomendoza/tutorial-xgboost-en-python-53e48fc58f73

In [ ]:
# Importamos la librería
import xgboost as xgb

In [ ]:
# Contamos los registros que contengan valor en la columna ['SalePrice] para diferenciar el dataframe train con el test
conteo_train_xgb = len(df_final_encode[(~df_final_encode.SalePrice.isnull())])

# Separando X_df_train_xgb e X_df_test_xgb
X_df_train_xgb = df_final_encode.iloc[:conteo_train_xgb,:]
y_df_train_xgb = X_df_train_xgb.SalePrice.values

X_df_test_xgb = df_final_encode.iloc[conteo_train_xgb:,:]
X_df_test_xgb_id = X_df_test_xgb.iloc[:,:1]

# Eliminamos la columna ['Id','SalePrice] de ambos dataframes
X_df_train_xgb.drop(['Id','SalePrice'], axis=1, inplace=True)
X_df_test_xgb.drop(['Id','SalePrice'], axis=1, inplace=True)

In [ ]:
# Creamos el objeto XGBRegressor y hacemos el split
modelo_xgb = xgb.XGBRegressor()
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X_df_train_ad, y_df_train_ad, test_size=0.33, random_state=42)

# Entrenamos el modelo XGBRegressor
modelo_xgb.fit(X_train_xgb, y_train_xgb)

# Predecimos sobre nuestro set de entrenamieto
y_train_pred_xgb = modelo_xgb.predict(X_train_xgb)

# Predecimos sobre nuestro set de test
y_test_pred_xgb = modelo_xgb.predict(X_test_xgb)


In [ ]:
# Imprimimos el la raíz cuadrada del error logarítmico medio
print('La raíz cuadrada del error logarítmico medio (RMSLE) sobre el dataset houses_train_raw es:', mean_squared_log_error(y_test_pred_xgb,y_test_xgb, squared=False))

In [ ]:
plt.scatter(y_test_pred_xgb, y_test_xgb, c="b", alpha=0.5)
plt.xlabel("y_test_pred_xgb")
plt.ylabel("y_test_xgb")
# plt.legend(loc='upper left')
plt.show()

In [ ]:
# Predecimos el target 'SalePrice' para el dataset houses_test_raw
xgb_preds_test=modelo_xgb.predict(X_df_test_xgb)
pd.DataFrame(xgb_preds_test, columns=['SalePrice']).to_csv("pred_test.csv", index=False, sep=",")

In [ ]:
pred_test_xgb = pd.DataFrame(xgb_preds_test, columns=['SalePrice'])

X_df_test_xgb_id.reset_index(drop=True, inplace=True)
pred_test_xgb.reset_index(drop=True, inplace=True)

pred_concat_xgb = pd.concat( [X_df_test_xgb_id, pred_test_xgb], axis=1) 

pred_concat_xgb.to_csv("pred_test.csv", index=False, sep=",")